# Acceleration with Google Coral


Although our code for Step 6 can achieve the basic function of counting squats, due to the heavy computation resources needed by Movenet, the operation is too slow, resulting in our counter running less smoothly and responsively.<br>
雖然我們Step6的代碼能夠實現深蹲計數的基本功能，但由於Movenet需要大量的計算資源，運行太過緩慢，使得我們的計數器運行不夠流暢和靈敏。<br>
<br>
The Coral accelerator is a hardware accelerator specifically designed to speed up deep learning inference, and can be used with Raspberry Pi 4. It can greatly improve the model inference speed, making it more smooth to run Movenet on Raspberry Pi 4.<br>
Coral accelerator是一種專門設計用於加速深度學習推理的硬體加速器，可以與Raspberry Pi 4一起使用。Coral accelerator可以大大提高模型推理速度，從而使得在Raspberry Pi 4上運行Movenet變得更加流暢。<br>

In [9]:
%run coral_common.ipynb

##### Next, let's try using Python to drive the Coral Accelerator to perform image inference.<br>接下來，讓我們試着用python驅動Coral Accelerator去執行圖片推理<br>
After importing the libraries, we defined function `get_result_from_image_with_model`. This function takes an image as input and applies a pre-trained pose detection model to extract the positions of key points on the human body in the image. With this function, we can quickly obtain MoveNet results using the Coral Accelerator by simply providing an image and an interpreter.<br>
導入相關庫之後，我們定義了`get_result_from_image_with_model`函數。這個函數需要輸入一張圖像並應用預先訓練的姿勢判斷模型，以提取圖像中人體關鍵點的位置。這個函數讓我們只需給定一張圖像和解釋器，就能夠利用Coral Accelerator快速得出MoveNet結果了。<br>

In [2]:
import PIL
from PIL import Image
from PIL import ImageDraw
from pycoral.adapters import common
from pycoral.utils.edgetpu import make_interpreter

interpreter = make_interpreter("movenet_single_pose_lightning_ptq_edgetpu.tflite")
interpreter.allocate_tensors()

_NUM_KEYPOINTS = 17

def get_result_from_image_with_model(input, interpreter, output=None):
    '''
    This function takes an input image and applies a pre-trained pose estimation model 
    to extract the position of human body keypoints in the image. 
    The TensorFlow Lite interpreter object is used to run inference on the input image
    using the pre-trained model.
    If the optional output argument is provided, the function also saves the input image
    with the detected keypoints drawn on it to the specified file path.

    Args:
    input: An input image file path or a numpy array representing an image.
    interpreter: A TensorFlow Lite interpreter object.
    output (optional): A file path to save the output image with keypoints drawn on it.

    Return:
    Array of keypoints. Each keypoint is represented as a triplet of [y, x, score].
    '''
    # load the input image using PIL
    if isinstance(input, str):
        img = PIL.Image.open(input)
    elif isinstance(input, np.ndarray):
        img = PIL.Image.fromarray(cv2.cvtColor(input, cv2.COLOR_BGR2RGB))

    # resize the image to model required size
    resized_img = img.resize(common.input_size(interpreter), PIL.Image.Resampling.LANCZOS)

    # load the resized image to interpreter
    common.set_input(interpreter, resized_img)

    # conduct the inference
    interpreter.invoke()

    # reshape and assign the inference result to variable `pose`
    keypoints_with_scores = common.output_tensor(interpreter, 0).copy().reshape(_NUM_KEYPOINTS, 3)

    # draw the keypoints and save the image (if specified `output`)
    if output:
        draw = ImageDraw.Draw(img)
        width, height = img.size
        for i in range(0, _NUM_KEYPOINTS):
            draw.ellipse(
            xy=[
                keypoints_with_scores[i][1] * width - 2, keypoints_with_scores[i][0] * height - 2,
                keypoints_with_scores[i][1] * width + 2, keypoints_with_scores[i][0] * height + 2
            ],
            fill=(255, 0, 0))
        img.save(output)
        
    return keypoints_with_scores

if __name__ == "__main__":
    keypoints_with_scores = get_result_from_image_with_model("input_image.jpeg", interpreter)
    print(type(keypoints_with_scores), keypoints_with_scores.shape, keypoints_with_scores, sep="\n\n")

<class 'numpy.ndarray'>

(17, 3)

[[0.32774833 0.58175325 0.36462   ]
 [0.3113609  0.589947   0.70056206]
 [0.3113609  0.5694627  0.5694627 ]
 [0.31955463 0.5735596  0.29907033]
 [0.3113609  0.50391304 0.49981618]
 [0.42197597 0.5571721  0.49981618]
 [0.40558854 0.43016967 0.49981618]
 [0.5121068  0.70875573 0.29907033]
 [0.5080099  0.53668785 0.8029834 ]
 [0.42197597 0.6841746  0.5694627 ]
 [0.4096854  0.65549666 0.19664899]
 [0.6022375  0.38100743 0.70056206]
 [0.6022375  0.28268293 0.70056206]
 [0.6473029  0.58175325 0.43016967]
 [0.6841746  0.41378227 0.43016967]
 [0.83166134 0.5162036  0.24581124]
 [0.88901734 0.33184516 0.75382113]]


Can this code produce the effect of Step6? Check the speed in the last cell.<br>
這個代碼能運行出Step6的效果嗎？在最後的單元格看看速度如何？

In [3]:
import time
time_elapsed = []

repetition = squat = squatting = 0
CONSECUTIVE_TH=3
def frame_process(frame):
    global repetition, squat, squatting
    
    # calculate and display angle
    start_time = time.time()
    #   ====== ⇩ ⇩ ⇩  write your answer here   ⇩ ⇩ ⇩ ======
    
    keypoints_with_scores = get_result_from_image_with_model(frame, interpreter)
    #  ====== ⇧ ⇧ ⇧  write your answer here  ⇧ ⇧ ⇧ ======
    time_elapsed.append( time.time() - start_time )
    
    hip_angle = calculate_hip_joint_angle(keypoints_with_scores)
    knee_angle = calculate_knee_joint_angle(keypoints_with_scores)

    # check if confidence is qualified, if yes, proceed to judgement
    if hip_angle is not None and knee_angle is not None:
        # if capture continuous CONSECUTIVE_TH angles in squatting, change squat to 1
        # if capture continuous CONSECUTIVE_TH angles in standing, change squat to 0 and repetition += 1
        if hip_angle<125 and knee_angle<105:
            if squat == 0:
                squatting += 1
            else:
                squatting = CONSECUTIVE_TH
        elif hip_angle>145 and knee_angle>125:
            if squat == 1:
                squatting -= 1
            else:
                squatting = 0
        if squatting == 0 and squat == 1:
            repetition += 1
            squat = 0
        elif squatting == CONSECUTIVE_TH and squat == 0:
            squat = 1
        text = f"Reps: {repetition}  Knee: {int(knee_angle)}  Hip: {int(hip_angle)}"
    else:
        text = f"Reps: {repetition}  Knee: ?  Hip: ?"
    if squat:
        text_color = (0,255,0)
    else:
        text_color = (128,192,64)
        
    putText(frame, text, text_color)
    return frame



In [8]:
stopButton = createStopButton()
display(stopButton)
thread = threading.Thread(target=view, args=(stopButton, "Produce_2_direct.mp4", 0, frame_process))
thread.start()

NameError: name 'createStopButton' is not defined

In [7]:

# Run
# ================
if __name__ == "__main__":
    display(stopButton)    
    thread = threading.Thread(target=view, args=(stopButton, 0, 1, frame_process))
    thread.start()

ToggleButton(value=True, button_style='danger', description='Stop', icon='square', tooltip='Description')

None

In [ ]:
average_time = sum(time_process) / len(time_process)

print(f"Average process time per frame is: { average_time }")
print(f"Average frame rate is: { 1 / average_time }")